In [1]:
%pip install happybase

Defaulting to user installation because normal site-packages is not writeable
     |################################| 40 kB 206 kB/s            
  Preparing metadata (setup.py) ... done
     |################################| 782 kB 282 kB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     |################################| 781 kB 215 kB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     |################################| 49 kB 256 kB/s            
  Created wheel for happybase: filename=happybase-1.2.0-py2.py3-none-any.whl size=23491 sha256=ec1963da8bf0d8fdb5741852c57bf8440de09a28e8b4879ecb1b388d78444b26
  Stored in directory: /home/itversity/.cache/pip/wheels/ee/78/c9/092e37a61812bae659556db5f294a

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType, DoubleType
from happybase import Connection
import sys
import traceback

In [35]:
spark = SparkSession.builder \
    .appName("KafkaStreamingToHBase") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.8,org.apache.hbase:hbase-client:2.4.9,org.apache.hbase:hbase-common:2.4.9") \
    .getOrCreate()

spark

In [36]:
# Define schema for the incoming JSON data
schema = StructType() \
    .add("eventType", StringType()) \
    .add("customerId", StringType()) \
    .add("productId", StringType()) \
    .add("timestamp", TimestampType()) \
    .add("metadata", StructType()
        .add("category", StringType())
        .add("source", StringType())
    ) \
    .add("quantity", IntegerType()) \
    .add("totalAmount", DoubleType()) \
    .add("paymentMethod", StringType()) \
    .add("recommendedProductId", StringType()) \
    .add("algorithm", StringType())


In [37]:
# Kafka connection details
bootstrap_servers = "pkc-56d1g.eastus.azure.confluent.cloud:9092"
kafka_topic = "Emad_topic" # add topic name
kafka_username = "JUKQQM4ZM632RECA"
kafka_password = "UUkrPuSttgOC0U9lY3ZansNsKfN9fbxZPFwrGxudDrfv+knTD4rCwK+KdIzVPX0D"

In [38]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config",
            f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{kafka_username}" password="{kafka_password}";') \
    .load()

In [39]:
json_df = df.selectExpr("CAST(value AS STRING)").select(from_json("value", schema).alias("data")).select("data.*")

In [40]:
def writeToHBase(df, epoch_id):
    print(f"Batch {epoch_id}: {df.count()} records")
    df.show(5, truncate=False)  # Show first 5 records

    rdd = df.rdd
    
    rdd.foreachPartition(lambda rows: writePartition(rows))

def writePartition(rows):
    
    hbase_host = 'hbase'
    hbase_table = 'CUSTEVENTS:CustomerEvents'
    
    try:
        connection = Connection(hbase_host)
        table = connection.table(hbase_table)
        print(f"Successfully connected to HBase and opened table {hbase_table}", file=sys.stderr)
        print(f"Available tables: {tables}", file=sys.stderr)
        
        if hbase_table.encode() not in tables:
            print(f"Table {hbase_table} does not exist!", file=sys.stderr)
            return
        
        row_count = 0
        for row in rows:
            try:
                row_key = row['customerId']
                data = row.asDict()
                hbase_data = {f'cf:{k}'.encode(): str(v).encode() for k, v in data.items()}
                table.put(row_key.encode(), hbase_data)
                row_count += 1
            except Exception as row_error:
                print(f"Error processing row: {str(row_error)}", file=sys.stderr)
                print(f"Row data: {row}", file=sys.stderr)
                traceback.print_exc(file=sys.stderr)
        
        print(f"Wrote {row_count} rows to HBase", file=sys.stderr)
        
    except Exception as e:
        print(f"Error in writePartition: {str(e)}", file=sys.stderr)
        traceback.print_exc(file=sys.stderr)
    finally:
        if 'connection' in locals():
            connection.close()

In [41]:
query = json_df \
    .writeStream \
    .foreachBatch(writeToHBase) \
    .outputMode("append") \
    .start()


In [42]:
query.awaitTermination()


Batch 0: 537 records
+-------------------+----------+---------+-------------------+---------------------+--------+-----------+-------------+--------------------+-------------+
|eventType          |customerId|productId|timestamp          |metadata             |quantity|totalAmount|paymentMethod|recommendedProductId|algorithm    |
+-------------------+----------+---------+-------------------+---------------------+--------+-----------+-------------+--------------------+-------------+
|productView        |68829     |3432     |2024-07-09 11:15:27|[Clothing, Search]   |null    |null       |null         |null                |null         |
|productView        |30516     |6488     |2024-07-09 11:15:29|[Electronics, Direct]|null    |null       |null         |null                |null         |
|recommendationClick|45014     |7822     |2024-07-09 11:15:31|[,]                  |null    |null       |null         |6637                |content_based|
|productView        |78939     |8505     |2024-07

StreamingQueryException: 'An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):\n  File "/opt/spark2/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 2381, in _call_proxy\n    return_value = getattr(self.pool[obj_id], method)(*params)\n  File "/opt/spark2/python/pyspark/sql/utils.py", line 191, in call\n    raise e\n  File "/opt/spark2/python/pyspark/sql/utils.py", line 188, in call\n    self.func(DataFrame(jdf, self.sql_ctx), batch_id)\n  File "<ipython-input-40-6e62a8a5909d>", line 7, in writeToHBase\n    rdd.foreachPartition(lambda rows: writePartition(rows))\n  File "/opt/spark2/python/pyspark/rdd.py", line 806, in foreachPartition\n    self.mapPartitions(func).count()  # Force evaluation\n  File "/opt/spark2/python/pyspark/rdd.py", line 1055, in count\n    return self.mapPartitions(lambda i: [sum(1 for _ in i)]).sum()\n  File "/opt/spark2/python/pyspark/rdd.py", line 1046, in sum\n    return self.mapPartitions(lambda x: [sum(x)])

In [1]:
spark.stop()

NameError: name 'spark' is not defined